Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-06 03:52:20--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.98.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.98.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  68.3MB/s    in 2.9s    

2019-11-06 03:52:23 (68.3 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [5]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


In [0]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
aisles = pd.read_csv('aisles.csv')
print(aisles.shape)
aisles.sample(4)

(134, 2)


,aisle_id,aisle
20,21,packaged cheese
98,99,canned fruit applesauce
63,64,energy sports drinks
18,19,oils vinegars


In [8]:
departments = pd.read_csv('departments.csv')
print(departments.shape)
departments.sample(4)

(21, 2)


,department_id,department
2,3,bakery
0,1,frozen
8,9,dry goods pasta
11,12,meat seafood


In [9]:
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.sample(4)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
7343939,775253,32360,2,1
32243786,3400970,10296,12,1
22005278,2321013,29487,4,1
3230759,341167,43555,4,1


In [10]:
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.sample(3)

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
1186801,2932543,17758,1,1
851364,2101864,18634,24,0
1146322,2831824,38656,2,1


In [11]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.sample(3)

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
403796,1083928,24289,prior,3,5,23,7.0
3412033,936197,205665,prior,13,2,12,9.0
574519,2791246,34594,prior,23,0,22,15.0


In [31]:
orders = orders[['order_id', 'user_id']]
orders

,order_id,user_id
0,2539329,1
1,2398795,1
2,473747,1
3,2254736,1
4,431534,1
...,...,...
3421078,2266710,206209
3421079,1854736,206209
3421080,626363,206209
3421081,2977660,206209


In [13]:
products= pd.read_csv('products.csv')
print(products.shape)
products.sample(3)

(49688, 4)


,product_id,product_name,aisle_id,department_id
22240,22241,Organic Garlic Roasted Garlic Pasta Sauce,9,9
25774,25775,Creamy Balsamic Yogurt Dressing,89,13
42882,42883,Round Top Natural Whole Grain Bread,112,3


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [16]:
print(order_products__prior.shape)
print(order_products__train.shape)

(32434489, 4)
(1384617, 4)


In [17]:
orders_products = pd.concat([order_products__prior, order_products__train], axis=0)
print(orders_products.shape)
orders_products.sample(3)

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
18678706,1970309,2242,1,1
28128947,2966283,44753,5,0
19558348,2062825,47042,3,0


In [18]:
orders_products = orders_products[['order_id','product_id']]
orders_products

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035
...,...,...
1384612,3421063,14233
1384613,3421063,35548
1384614,3421070,35951
1384615,3421070,16953


In [32]:
print(orders_products.columns)
print(orders.columns)

Index(['order_id', 'product_id'], dtype='object')
Index(['order_id', 'user_id'], dtype='object')


In [21]:
print(orders_products.describe())

           order_id    product_id
count  3.381911e+07  3.381911e+07
mean   1.710566e+06  2.557551e+04
std    9.874008e+05  1.409770e+04
min    1.000000e+00  1.000000e+00
25%    8.554130e+05  1.351900e+04
50%    1.710660e+06  2.525600e+04
75%    2.565587e+06  3.793500e+04
max    3.421083e+06  4.968800e+04


In [33]:
orders_full = pd.merge(orders_products, orders, how='left', on='order_id')
orders_full

,order_id,product_id,user_id
0,2,33120,202279
1,2,28985,202279
2,2,9327,202279
3,2,45918,202279
4,2,30035,202279
...,...,...,...
33819101,3421063,14233,169679
33819102,3421063,35548,169679
33819103,3421070,35951,139822
33819104,3421070,16953,139822


In [0]:
product_names = ['Banana',
'Bag of Organic Bananas',
'Organic Strawberries',
'Organic Baby Spinach',
'Organic Hass Avocado',
'Organic Avocado',
'Large Lemon',
'Strawberries',
'Limes',
'Organic Whole Milk']

In [24]:
condition = products['product_name'].isin(product_names)
products = products[condition]
products

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
24851,24852,Banana,24,4
26208,26209,Limes,24,4
27844,27845,Organic Whole Milk,84,16
47208,47209,Organic Hass Avocado,24,4
47625,47626,Large Lemon,24,4
47765,47766,Organic Avocado,24,4


In [25]:
products = products[['product_id', 'product_name']]
products

,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


In [34]:
full_list = (products[['product_id', 'product_name']]
             .merge(orders_full[['product_id', 'order_id']]))
full_list

,product_id,product_name,order_id
0,13176,Bag of Organic Bananas,5
1,13176,Bag of Organic Bananas,27
2,13176,Bag of Organic Bananas,29
3,13176,Bag of Organic Bananas,32
4,13176,Bag of Organic Bananas,42
...,...,...,...
2418309,47766,Organic Avocado,3419827
2418310,47766,Organic Avocado,3420232
2418311,47766,Organic Avocado,3420257
2418312,47766,Organic Avocado,3420578


In [35]:
full_list['product_name'].value_counts(ascending=False)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [0]:
##### YOUR CODE HERE #####

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####